In [3]:
from torch import nn
from torch.nn.functional import relu
import torch
import os
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from lib.models import Gandalf
from lib.utils import *

from torch.utils.data import DataLoader
from sklearn.metrics import ConfusionMatrixDisplay,classification_report

In [15]:
FOLD = 0

folds = get_leave_one_out_cv_ids_for_ekyn()
train_ids,test_ids = folds[FOLD]
X,y = load_eeg_label_pair(id=train_ids[0],condition='PF')
testloader = DataLoader(Windowset(X,y),batch_size=32,shuffle=False)
print(f'testloader: {len(testloader)} batches')

model = Gandalf()
model.load_state_dict(torch.load(f=f'{GANDALFS_PATH}/gandalf_{FOLD}/last_model.pt',map_location='cpu'))
model.to(DEVICE)

criterion = torch.nn.CrossEntropyLoss()
loss,y_true,y_pred = evaluate(testloader,model,criterion,'cuda')
ConfusionMatrixDisplay.from_predictions(y_true,y_pred)
report = classification_report(y_true,y_pred,output_dict=True)

/tmp/ipykernel_879779/2017346022.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f=f'{GANDALFS_PATH}/gandalf_{FOLD}/last_model.pt',map_

testloader: 270 batches


 41%|████      | 111/270 [00:04<00:06, 24.30it/s]

In [13]:
from sklearn.metrics import ConfusionMatrixDisplay,classification_report

folds = get_leave_one_out_cv_ids_for_ekyn()
criterion = torch.nn.CrossEntropyLoss()
reports = []

for FOLD,(train_ids,test_ids) in tqdm(enumerate(folds)):
    model = Gandalf()
    model.load_state_dict(torch.load(f=f'{GANDALFS_PATH}/gandalf_{FOLD}/best_model.pt',map_location='cpu',weights_only=False))
    model.to(DEVICE)
    for condition in ['Vehicle','PF']:
        X,y = load_eeg_label_pair(id=test_ids[0],condition=condition)
        testloader = DataLoader(Windowset(X,y),batch_size=32,shuffle=False)
        loss,y_true,y_pred = evaluate(testloader,model,criterion,'cuda')
        report = classification_report(y_true,y_pred,output_dict=True)
        reports.append(report)

0it [00:00, ?it/s]/tmp/ipykernel_879082/573782102.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f=f'{GANDALFS_PATH}/gandalf_{FOLD}/bes

In [17]:
[report['macro avg']['f1-score'] for report in reports]

[0.7497158451653374,
 0.7076714266654877,
 0.697565175625257,
 0.669308845133585,
 0.730688764572463,
 0.7454268895381664,
 0.7263209654624437,
 0.7366998788175567,
 0.724037608298091,
 0.6891391600753284,
 0.6618482017433407,
 0.5479359077335869,
 0.7013755863201446,
 0.7156419832181813,
 0.6864939527264952,
 0.6989276761877612,
 0.7014350779884696,
 0.7030651600632591,
 0.6931989548124466,
 0.7102414916358168,
 0.6818893963497358,
 0.6714282887079445,
 0.6834193765549834,
 0.675662099798097,
 0.7368521473437685,
 0.728086486477015,
 0.6939968400587316,
 0.7096473501308634,
 0.6352741637010335,
 0.6744911611440095,
 0.6828716126577463,
 0.6539611354388838]